<a href="https://colab.research.google.com/github/yunjeong-chang/My_Data_project/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98/%EC%8B%AC%EB%A6%AC%20%EC%84%B1%ED%96%A5%20%EC%98%88%EC%B8%A1(%EB%AA%A8%EB%8D%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
# 전처리 완료된 데이터 불러오기 
final_train_x = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/심리_성향_예측_Data/심리 성향 예측_final_train_x.csv')
final_train_y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/심리_성향_예측_Data/심리 성향 예측_final_train_y.csv')
final_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/심리_성향_예측_Data/심리 성향 예측_final_test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/심리_성향_예측_Data/sample_submission.csv')

In [ ]:
print(final_train_x.shape)
print(final_train_y.shape)
print(final_test.shape)
print(sample_submission.shape)

(45532, 77)
(45532, 1)
(11383, 77)
(11383, 2)


### **1. 간단한 LGBM 모델**

public(0.5788903241) 

private(0.5750718696)

참고 : [코드 공유](https://dacon.io/competitions/official/235647/codeshare/1700?page=1&dtype=recent)


먼저 베이스라인 코드로 제시된 글에서 쓰인 모델을 써봤습니다.

**Light GBM**은 Tree가 수직적으로 확장(leaf-wise)되는 반면에, 다른 알고리즘은 Tree가 수평적으로 확장(level-wise)됩니다. 

즉, 동일한 leaf를 확장할 때 Light GBM은 더 많은 손실(loss)을 줄일 수 있습니다.

- 장점 : 큰 사이즈의 데이터를 다룰 때 속도가 빠르고, 실행 시 적은 메모리를 차지합니다.

- 단점 : 과적합에 민감하여 작은 사이즈의 데이터에는 적합하지 않습니다.

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(n_estimators=500) # n_estimators는 너무 커지면 과적합이 발생합니다. 우선은 다른 파라미터는 건드리지 않았습니다.
model.fit(final_train_x, final_train_y)  # 모델 학습
final_pred = model.predict(final_test)   # 예측
sample_submission['voted']=final_pred    # 저장
sample_submission.to_csv('test_submit_LGBM_basic.csv', index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### **2. NN 모델**

제출 횟수 제한 때문에 다양한 파라미터 실험은 못해봤지만, 비교적 성능이 안좋아서 제외 시켰습니다.

하지만 코드 공유에서 대다수를 차지하는 보편적인 모델입니다.

### **3. CAT+LGBM+XGB 앙상블 모델**

public(0.5890179708)

private(0.5922709518)

생각보다는 좋지 않았지만 그래도 private에서 점수가 더 높아지는 것을 보니 앙상블 한 것이 효과를 본 것 같다.

참고 : [코드 공유](https://dacon.io/competitions/official/235647/codeshare/1821?page=1&dtype=recent)

코드 공유 중에 부스팅 계열의 세가지 모델을 앙상블 한 것이 있어서 사용해 봤습니다.

**XGBoost Classifier + Light GBM Classifier + CatBoostClassifier**

- XGBoost Classifier는 분류 정확도는 우수하나, Outlier에 취약합니다. 과적합 방지 기능을 하는 규제가 포함 되어 있습니다.

- Light GBM Classifier는 위의 베이스라인 코드에서 사용한 모델입니다.

- CatBoostClassifier는 기본 파라미터 최적화가 잘 되어있어 두 모델에 비해서는 복잡한 하이퍼 파라미터 튜닝을 필요로 하지는 않습니다. 학습 데이터에 대한 샘플링을 통해 모델을 만들고, 샘플링이 되지 않은 데이터의 잔차를 추정하면서 학습 합니다.

In [ ]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import xgboost
from xgboost import plot_importance
from xgboost import XGBClassifier

In [ ]:
# 모델에 넣을 하이퍼파라미터를 미리 설정해줍니다. 
params =  {'colsample_bylevel': 0.92859, 'colsample_bytree': 0.55352, 'eta': 0.015, 'gamma': 0.62587, 'min_child_weight': 48.0, 'reg_alpha': 0.54, 'reg_lambda': 1.062, 'subsample': 0.89}  
params['min_child_weight'] = int(params['min_child_weight'])

model = XGBClassifier(
             **params,
             max_depth=11,
             booster='gbtree',
             n_estimators=5000,
             objective= 'binary:logistic',
             eval_metric='auc',
             n_jobs= -1,
             scale_pos_weight= 1.206,
             random_state= 55 
)

# XGBoost 분류 모델로 학습 및 예측
model.fit(final_train_x, final_train_y)
XGBC_final_pred = model.predict(final_test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# 모델에 넣을 하이퍼파라미터를 미리 설정해줍니다. 
params = {'colsample_bytree': 0.675479, 'learning_rate': 0.00645, 'max_depth': 10.3, 'min_child_samples': 88.41, 'min_child_weight': 28.4, 'min_split_gain': 0.025029, 'num_leaves': 86.2, 'reg_alpha': 0.544736, 'reg_lambda': 0.15015, 'subsample': 0.7295}

params['num_leaves'] = int(params['num_leaves'])
params['max_depth'] = int(params['max_depth'])
params['min_child_samples'] = int(params['min_child_samples'])

model = LGBMClassifier(
        **params,
        objective= 'binary',
        subsample_for_bin= 240000,
        is_unbalance= False,
        n_estimators=10000,
        n_jobs=-1,
        silent= -1,
        verbose= -1,
        random_state=55     
)

# LGBM 분류 모델로 학습 및 예측
model.fit(final_train_x, final_train_y)
LGBM_final_pred = model.predict(final_test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# 모델에 넣을 하이퍼파라미터를 미리 설정해줍니다. 
params = {'bagging_temperature': 0.375906, 'depth': 9.0, 'l2_leaf_reg': 68.8, 'learning_rate': 0.011, 'od_wait': 138.699, 'subsample': 0.76046}

params['depth'] = int(params['depth'])
params['l2_leaf_reg'] = int(params['l2_leaf_reg'])
params['od_wait'] = int(params['od_wait'])

model = CatBoostClassifier(
                          **params,
                          iterations=5000,
                          eval_metric='AUC',
                          allow_writing_files=False,
                          od_type='Iter',
                          random_state=55)

# CatBoost 분류 모델로 학습 및 예측
model.fit(final_train_x, final_train_y)
CatBoost_final_pred = model.predict(final_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0:	total: 174ms	remaining: 14m 28s
1:	total: 259ms	remaining: 10m 47s
2:	total: 342ms	remaining: 9m 29s
3:	total: 425ms	remaining: 8m 51s
4:	total: 510ms	remaining: 8m 29s
5:	total: 596ms	remaining: 8m 15s
6:	total: 680ms	remaining: 8m 5s
7:	total: 765ms	remaining: 7m 57s
8:	total: 855ms	remaining: 7m 54s
9:	total: 946ms	remaining: 7m 51s
10:	total: 1.03s	remaining: 7m 46s
11:	total: 1.11s	remaining: 7m 40s
12:	total: 1.2s	remaining: 7m 41s
13:	total: 1.29s	remaining: 7m 38s
14:	total: 1.37s	remaining: 7m 35s
15:	total: 1.46s	remaining: 7m 34s
16:	total: 1.54s	remaining: 7m 32s
17:	total: 1.63s	remaining: 7m 29s
18:	total: 1.71s	remaining: 7m 28s
19:	total: 1.79s	remaining: 7m 27s
20:	total: 1.87s	remaining: 7m 24s
21:	total: 1.96s	remaining: 7m 23s
22:	total: 2.04s	remaining: 7m 21s
23:	total: 2.12s	remaining: 7m 19s
24:	total: 2.21s	remaining: 7m 19s
25:	total: 2.29s	remaining: 7m 18s
26:	total: 2.37s	remaining: 7m 17s
27:	total: 2.47s	remaining: 7

In [ ]:
# 세 모델에서 얻은 예측값들에 적당한 가중치를 주어 앙상블 합니다.
ensemble_preds = (CatBoost_final_pred * 0.55) + (XGBC_final_pred * 0.37) + (LGBM_final_pred * 0.08)

#앙상블 한 예측값을 저장합니다.
sample_submission['voted']= ensemble_preds
sample_submission['voted'] = sample_submission['voted'].astype('int') #결과가 실수 형태로 나오는 것이 있어 int형으로 바꿔줍니다.
sample_submission.to_csv('test_submit_CAT+XGB+LGBM.csv', index=False)

### **4. Auto ML 모델**

public(0.6220532941) 657등 (0.7대가 나오지 않아서 아쉽다 ㅠ)

private(0.6227340114)

참고 : [코드 공유](https://dacon.io/competitions/official/235647/codeshare/1720?page=3&dtype=recent)



**Auto ML** 분야는 여러 종류가 있는데요. 

그 중 Hyperparameter Optimization 분야에서 **Bayesian Optimization**(베이지안 최적화, 베이즈 최적화)를 사용한 코드가 있어 참고해 봤습니다.

Bayesian Optimization는 하이퍼 파라미터 튜닝에서 기존의 grid search, random search를 대체하는데 사용 가능합니다.

즉, 기본 모델로는 LGBM을 사용하고 그 최적의 하이퍼파라미터 값을 **Bayesian Optimization** 방법으로 찾아내는 모델입니다!

In [ ]:
#BayesianOptimization를 import 하기위해 필요한 설치 과정
pip install bayesian-optimization 

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=ad9b7e79c05ca5a0bce82fa92354cd4d79ea0144c60395b821972e459b11f18f
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
import lightgbm as lgbm
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_validate

In [ ]:
# 목적 함수 생성
# 목적 함수란 사용할 LGBMClassifier 모델의 성능 함수로 모델의 파라미터 조합을 입력값으로 가집니다. 성능평가는 auc이므로 score값을 return 해 줍니다.
# 최적의 값을 탐색하고자 하는 파리미터를 함수에 입력해줍니다.

def lgbm_cv(learning_rate, num_leaves, max_depth, min_child_weight, colsample_bytree, feature_fraction, bagging_fraction, lambda_l1, lambda_l2):
    model = lgbm.LGBMClassifier(learning_rate=learning_rate,
                                n_estimators = 300,
                                num_leaves = int(round(num_leaves)),
                                max_depth = int(round(max_depth)),
                                min_child_weight = int(round(min_child_weight)),
                                colsample_bytree = colsample_bytree,
                                feature_fraction = max(min(feature_fraction, 1), 0),
                                bagging_fraction = max(min(bagging_fraction, 1), 0),
                                lambda_l1 = max(lambda_l1, 0),
                                lambda_l2 = max(lambda_l2, 0)
                               )
    scoring = {'roc_auc_score': make_scorer(roc_auc_score)}
    result = cross_validate(model, final_train_x, final_train_y, cv=5, scoring=scoring)
    auc_score = result["test_roc_auc_score"].mean()
    return auc_score

In [ ]:
# 파라미터의 탐색 대상 구간을 설정 합니다.
pbounds = {'learning_rate' : (0.0001, 0.05),    # learning_rate : 보통 0.01~ 정도로 설정합니다. 세부 조정을 위해서는 0.0001~정도로 설정해도 무방합니다.
           'num_leaves': (300, 600),            # num_leaves : 250정도로 설정해도 무방합니다. 300~600 정도로 설정했습니다.
           'max_depth': (2, 25),                # max_depth : -1 로 설정하면 무한대로 트리가 길어집니다. 9~ 정도로 설정하는게 무방하나 조금 더 넓은 범위로 설정했습니다.
           'min_child_weight': (30, 100),        
           'colsample_bytree': (0, 0.99),
           'feature_fraction': (0.0001, 0.99),  # feature_fraction, bagging_fraction : 0과 1 사이의 범위로 설정했습니다.
           'bagging_fraction': (0.0001, 0.99),
           'lambda_l1' : (0, 0.99),
           'lambda_l2' : (0, 0.99),
          }

In [ ]:
#객체 생성
lgbmBO = BayesianOptimization(f = lgbm_cv, pbounds = pbounds, verbose = 2, random_state = 0 ) #f는 목적 함수, pbounds는 입력값의 탐색 구간

In [ ]:
# 반복적으로 베이지안 최적화 수행
lgbmBO.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)

|   iter    |  target   | baggin... | colsam... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

|  1        |  0.6853   |  0.5434   |  0.708    |  0.5968   |  0.5394   |  0.4194   |  0.03233  |  12.06    |  92.42    |  589.1    |


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

|  2        |  0.6843   |  0.3797   |  0.7838   |  0.5237   |  0.5624   |  0.9163   |  0.003645 |  4.004    |  31.42    |  549.8    |


...

|  23       |  0.6607   |  0.0001   |  0.0      |  0.99     |  0.99     |  0.0      |  0.05     |  2.0      |  30.0     |  364.0    |


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

|  24       |  0.6882   |  0.0001   |  0.0      |  0.99     |  0.99     |  0.0      |  0.05     |  2.0      |  88.66    |  311.7    |


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

|  25       |  0.6821   |  0.99     |  0.0      |  0.99     |  0.99     |  0.0      |  0.05     |  25.0     |  100.0    |  368.2    |


In [ ]:
# 찾은 파라미터 값 확인
lgbmBO.max

{'params': {'bagging_fraction': 0.4729589655278779,
  'colsample_bytree': 0.6158738007642527,
  'feature_fraction': 0.8182268598478806,
  'lambda_l1': 0.7594444092904203,
  'lambda_l2': 0.6899513386599342,
  'learning_rate': 0.015987379446646303,
  'max_depth': 3.1851686676923165,
  'min_child_weight': 98.03887622366321,
  'num_leaves': 301.8072068585756},
 'target': 0.6935909482114496}

In [ ]:
#최적의 파라미터 적용
fit_lgbm = lgbm.LGBMClassifier(learning_rate=lgbmBO.max['params']['learning_rate'],
                               num_leaves = int(round(lgbmBO.max['params']['num_leaves'])),
                               max_depth = int(round(lgbmBO.max['params']['max_depth'])),
                               min_child_weight = int(round(lgbmBO.max['params']['min_child_weight'])),
                               colsample_bytree=lgbmBO.max['params']['colsample_bytree'],
                               feature_fraction = max(min(lgbmBO.max['params']['feature_fraction'], 1), 0),
                               bagging_fraction = max(min(lgbmBO.max['params']['bagging_fraction'], 1), 0),
                               lambda_l1 = lgbmBO.max['params']['lambda_l1'],
                               lambda_l2 = lgbmBO.max['params']['lambda_l2']
                               )

In [ ]:
# 이제 똑같이 모델 학습 및 예측을 진행 해 줍니다!
model = fit_lgbm.fit(final_train_x,final_train_y)
final_pred = model.predict(final_test)
sample_submission['voted']=final_pred
sample_submission.to_csv('test_submit_AUTO_ML.csv', index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
